# Recuperação de dados no Twitter

Este notebook tem como objetivo demostrar a recuperação de dados no tweeter, via API e persistir as informações em documentos para posterior uso.

## Imports

Vamos utilizar nesse exemplo as bibliotecas:

* os: Para acesso aos recursos do sistema operacional
* tweepy: Para acesso ao twitter (login, coleta)
* pandas: Para manipulação dos dados no formato tabular
* sklearn.*: Para processamento de machine learning e tratamento de dados
* spacy: Para processamento de linguagem natural
* matplotlib: Para renderização de graficos

In [56]:
import os
import nltk
import spacy
import tweepy
import wordcloud
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

## Download do idioma

Vamos fazer o download do idioma para manipulação dos textos

In [2]:
# !python -m spacy download pt
# !python -m spacy download pt_core_news_sm

In [34]:
nlp = spacy.load('pt_core_news_sm')

## Carregamento das chaves de acesso

As chaves de acesso podem ser geradas através do portal do desenvolvedor do Twitter.

* Site do Twitter: https://twitter.com/ (Links para um site externo.)

* Área de desenvolvedor https://developer.twitter.com/en/apply/user (Links para um site externo.)

* Área de aplicativos https://developer.twitter.com/en/apps (Links para um site externo.)


In [4]:
consumer_key =  os.environ['tweety_consumer_key']
consumer_secret = os.environ['tweety_consumer_secret']
access_token = os.environ['tweety_access_token']
acess_token_secret = os.environ['tweety_acess_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, acess_token_secret)
api = tweepy.API(auth)

## Dataframe

Vamos utilizar um dataframe para persistir nossos registros obtidos a partir do stream de twetts. O Dataframe é similar a uma planilha.

In [5]:
df = pd.DataFrame(columns=['name', 'description', 'created_at', 'text', 'location', 'retweeted', 'owner_name', 'owner_location', 'owner_created_at', 'retweeted_count'])
df.head()

,name,description,created_at,text,location,retweeted,owner_name,owner_location,owner_created_at,retweeted_count


## Streamlister - Execute esse bloco se você não tiver o dataset salvo ainda

Vamos criar um stream listener que ira receber os twitts de acordo com a pesquisa que fizermos. Para isso vamos extender uma classe do pacote tweepy e fazer nossa implementação.

In [6]:
class StreamListner(tweepy.StreamListener):
    def __init__(self, df):
        super().__init__()
        self.__df__ = df

    def on_status(self, status):
        json = dict(status._json)
        user = json['user']
        
        retweety = json.get('retweeted_status')
        retweety_user = retweety['user'] if retweety != None and retweety.get('user') != None else None

        obj = { 'name': user['name'], 'description': user['description'], 'created_at': json['created_at'], 'text': json['text'], 'location': user.get('location'), 'retweeted': retweety != None }

        if retweety != None:
            obj['owner_name'] = retweety_user['name']
            obj['owner_location'] = retweety_user.get('location')
            obj['owner_created_at'] = retweety['created_at']
            obj['retweeted_count'] = retweety.get('retweet_count', 0)
        
        self.__df__ = self.__df__.append(obj, ignore_index=True)
        

    def on_error(self, status_code):
        if status_code == 420:
            return False

    @property
    def df(self):
        return self.__df__

### Definir o(s) termo(s) de pesquisa.

Podemos definir os n termos que serão utilizados para pesquisa

In [7]:
track = ['covid19', 'covid-19', 'sars-cov-2', 'coronavirus', 'corona-virus']

Criamos o objeto responsável pela busca dos tweets

In [8]:
stream_listner = StreamListner(df)
stream = tweepy.Stream(auth=api.auth, listener=stream_listner)
stream.filter(track=track, languages=['pt'], is_async=True)

In [18]:
stream.disconnect()

## Visualização dos registros - Obtidos pelo stream

Vamos visualizar os registros obtidos pelo stream atraves do dataframe

In [19]:
df = stream_listner.df

In [20]:
df

,name,description,created_at,text,location,retweeted,owner_name,owner_location,owner_created_at,retweeted_count
0,Edemar Motta,"Deus, ajude-me a ser a pessoa que meu cachorro...",Thu Jan 21 13:48:54 +0000 2021,@UOLNoticias Ou fecha tudo por um longo períod...,"Casimiro de Abreu, RJ",False,NaN,NaN,NaN,NaN
1,A Crítica,"Para quem quer notícia todo dia, o dia todo. \...",Thu Jan 21 13:48:54 +0000 2021,Defensoria do AM pede que prefeitura e Estado ...,"Manaus, Amazonas",False,NaN,NaN,NaN,NaN
2,Renato mais mascarado,A conta para acabar com todas as contas • Polí...,Thu Jan 21 13:48:55 +0000 2021,"RT @vinnybrandt: Maluco, o tamanho do pacotão ...","São Paulo, Brasil",True,Vinícius Sacramento,Rio de Janeiro,Thu Jan 21 13:48:09 +0000 2021,1
3,Melting,None,Thu Jan 21 13:48:57 +0000 2021,"RT @oatila: Quando faltar vacina, insumo para ...",None,True,Atila Iamarino,São Paulo,Tue Jan 19 17:44:38 +0000 2021,3206
4,ʟ ᴇ ᴏ,Se uma batata consegue tornar-se vodka então e...,Thu Jan 21 13:48:59 +0000 2021,Especialmente com a nova estirpe inglesa de CO...,Stu(dying) @ FMUP,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
115,Clayton Lula da Silva 🇦🇲🇦🇲🇦🇲🇦🇲🇦🇲🇦🇲🇦🇲,http://instagram.com/clayton362,Thu Jan 21 13:50:18 +0000 2021,RT @JFMargarida: A insistência no uso da cloro...,"Belo Horizonte, Brasil",True,Margarida Salomão,Juiz de Fora,Thu Jan 21 13:07:30 +0000 2021,16
116,Rosana Lancsarics,Journalist/Public Relations. Love sports: Mara...,Thu Jan 21 13:50:18 +0000 2021,RT @LucianoHuck: Deixo aqui meu carinho a famí...,Toronto,True,Luciano Huck,"Rio de Janeiro, Brasil",Thu Jan 21 13:48:49 +0000 2021,2
117,Nilson Braga,None,Thu Jan 21 13:50:19 +0000 2021,RT @Poder360: Incêndio atinge laboratório que ...,None,True,Poder360,"Brasília, Brasil",Thu Jan 21 13:30:53 +0000 2021,4
118,G1 - São Paulo,Acompanhe as notícias de São Paulo no G1,Thu Jan 21 13:50:19 +0000 2021,Estado de SP tem 58 municípios com mais de 80%...,São Paulo/SP,False,NaN,NaN,NaN,NaN
